<a href="https://colab.research.google.com/github/MathurUtkarsh/Fake_News_LSTM/blob/main/Fake_News_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Fake News Classifier Using LSTM

### Data Loading

In [2]:
import pandas as pd

In [9]:
df = pd.read_csv('/content/train.csv', quoting=0, error_bad_lines = False, encoding = 'utf-8')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


In [10]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [11]:
df = df.dropna()

In [12]:
X = df.drop('label', axis = 1)

In [13]:
y = df['label']

In [14]:
X.shape

(18285, 4)

In [15]:
y.shape

(18285,)

In [16]:
import tensorflow as tf

In [17]:
tf.__version__

'2.8.2'

In [28]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [18]:
### Vocabulary size
voc_size=5000

### Onehot Representation

In [19]:
messages=X.copy()

In [20]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [21]:
messages.reset_index(inplace=True)

In [22]:
import nltk
import re
from nltk.corpus import stopwords

In [23]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

### Data Preprocessing

In [25]:
# Function for stemming and stop words.
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    print(i)
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

Streaming output truncated to the last 5000 lines.
13285
13286
13287
13288
13289
13290
13291
13292
13293
13294
13295
13296
13297
13298
13299
13300
13301
13302
13303
13304
13305
13306
13307
13308
13309
13310
13311
13312
13313
13314
13315
13316
13317
13318
13319
13320
13321
13322
13323
13324
13325
13326
13327
13328
13329
13330
13331
13332
13333
13334
13335
13336
13337
13338
13339
13340
13341
13342
13343
13344
13345
13346
13347
13348
13349
13350
13351
13352
13353
13354
13355
13356
13357
13358
13359
13360
13361
13362
13363
13364
13365
13366
13367
13368
13369
13370
13371
13372
13373
13374
13375
13376
13377
13378
13379
13380
13381
13382
13383
13384
13385
13386
13387
13388
13389
13390
13391
13392
13393
13394
13395
13396
13397
13398
13399
13400
13401
13402
13403
13404
13405
13406
13407
13408
13409
13410
13411
13412
13413
13414
13415
13416
13417
13418
13419
13420
13421
13422
13423
13424
13425
13426
13427
13428
13429
13430
13431
13432
13433
13434
13435
13436
13437
13438
13439
13440
13441
13442
1

In [26]:

corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [29]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr

[[2012, 2203, 4757, 318, 2441, 1692, 171, 3183, 2832, 3232],
 [1698, 3887, 2051, 128, 4754, 376, 1160],
 [2344, 4551, 3063, 1104],
 [2179, 913, 3812, 2926, 182, 2934],
 [2893, 4754, 1693, 2597, 2685, 1788, 4754, 819, 4162, 4774],
 [2881,
  4829,
  255,
  3442,
  3499,
  2157,
  1957,
  1461,
  4793,
  1916,
  2283,
  3581,
  3748,
  2298,
  1160],
 [4134, 4525, 4540, 1059, 3137, 2521, 1578, 4264, 2193, 2238, 2020],
 [1448, 988, 2772, 4710, 1946, 4519, 2157, 1250, 2193, 2238, 2020],
 [61, 604, 1500, 2271, 2778, 125, 2888, 2409, 2157, 1916],
 [3194, 461, 2801, 2913, 951, 2459, 606, 3289],
 [4847, 2616, 4237, 1120, 1383, 1948, 4680, 474, 2520, 1017, 2992],
 [2926, 4470, 2441, 125, 2157, 1946],
 [136, 304, 2634, 4409, 822, 1684, 3617, 1890, 4498],
 [4919, 872, 2299, 893, 1188, 223, 2319, 2193, 2238, 2020],
 [3365, 4386, 2761, 2282, 3505, 2193, 2238, 2020],
 [670, 2, 2740, 3908, 4887, 2129, 2277, 4033, 4627, 1759],
 [4591, 4210, 3887],
 [906, 3263, 3994, 3221, 2157, 4779, 1585, 1160],
 [434

### Embedding Representation

In [30]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 3183 2832 3232]
 [   0    0    0 ... 4754  376 1160]
 [   0    0    0 ... 4551 3063 1104]
 ...
 [   0    0    0 ... 2193 2238 2020]
 [   0    0    0 ... 1730 4505 4109]
 [   0    0    0 ...  885 1890 4741]]


In [31]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 2012,
       2203, 4757,  318, 2441, 1692,  171, 3183, 2832, 3232], dtype=int32)

### LSTM Model

In [32]:
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [33]:
len(embedded_docs),y.shape

(18285, (18285,))

### Create new feature and label

In [34]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [35]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [36]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

### Model Training

In [37]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 9s 9ms/step - loss: 0.3372 - accuracy: 0.8422 - val_loss: 0.1941 - val_accuracy: 0.9176
Epoch 2/10
192/192 [==============================] - 1s 7ms/step - loss: 0.1416 - accuracy: 0.9419 - val_loss: 0.1987 - val_accuracy: 0.9236
Epoch 3/10
192/192 [==============================] - 1s 7ms/step - loss: 0.0992 - accuracy: 0.9641 - val_loss: 0.2207 - val_accuracy: 0.9206
Epoch 4/10
192/192 [==============================] - 1s 7ms/step - loss: 0.0735 - accuracy: 0.9740 - val_loss: 0.2496 - val_accuracy: 0.9165
Epoch 5/10
192/192 [==============================] - 1s 7ms/step - loss: 0.0481 - accuracy: 0.9833 - val_loss: 0.3002 - val_accuracy: 0.9175
Epoch 6/10
192/192 [==============================] - 2s 8ms/step - loss: 0.0305 - accuracy: 0.9899 - val_loss: 0.3664 - val_accuracy: 0.9094
Epoch 7/10
192/192 [==============================] - 1s 7ms/step - loss: 0.0225 - accuracy: 0.9932 - val_loss: 0.3236 - val_accuracy: 0.9148
Epoch 

### Adding Dropout

In [54]:
from tensorflow.keras.layers import Dropout

In [95]:
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(500))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])